In [1]:
from motion_planner import MotionPlanner, Trajectory, RobotModel
import numpy as np

### Create your motion planner

In [2]:
planner = MotionPlanner(RobotModel.Panda)

link name in Motion planner constructor panda_tool
panda_tool
urdf : descriptions/robot_descriptions/franka_panda_bullet/panda_arm.urdf


### Set boundary conditions

In [3]:
# Initial state
q0 = np.array([ 0.67373054,  1.22464762, -0.41799474, -0.9453,     -0.69665789,  1.42159091, -0.990211  ])
dq0 = np.array([-0.62467405, -0.36846654, -0.64362514,  0.84445767, -0.59795651,  1.08140881,  0.        ])
ddq0 = np.zeros_like(q0)

qd = 1.2 * q0 # No meaning, just use whatever you want
dqd = 0.8 * dq0
ddqd = np.zeros_like(qd)

planner.set_current_state((q0, dq0, ddq0))
planner.set_target_state((qd, dqd, ddqd))

### Solve and get trajectory

In [4]:
planner.solve() # You can specify other parameters such as sqp_max_iter
traj = planner.get_trajectory()
traj_ruckig = planner.get_trajectory(ruckig=True)

Before mpc.solve()
After mpc.solve()


If you have several trajectories to compute, you can concatenate these by using the "+" operator :

In [5]:
print("Shape of traj is initially : {} ({} trajectory of {} time-steps)".format(traj.shape, *traj.shape))
traj2 = planner.get_trajectory()
traj += traj2
print("After using the + operator traj has now shape : {} ({} trajectories of {} time-steps each)".format(traj.shape, *traj.shape))

Shape of traj is initially : (1, 101) (1 trajectory of 101 time-steps)
After using the + operator traj has now shape : (2, 101) (2 trajectories of 101 time-steps each)


### Accessing t, q, qdot, qddot and tau
To access the results, you can use one of these notations:

In [10]:
t = traj["t"]
q = traj.q
qdot = traj["qdot"][0] # If you want to access the first q trajectory
t, q, qdot, qddot, tau = traj[1] # If you want to get all the state from the second trajectory at once

# 
# etc ..
#

print(t.shape, q.shape, qdot.shape, qddot.shape, tau.shape)

(101,) (101, 7) (101, 7) (101, 7) (101, 7)


### Trajectory analysis
The .get_trajectory() method returns a Trajectory object that provides several useful analysis methods. To check if the trajectories satasfy all the constraints: 

In [6]:
print("Constraint satisfied : {}".format(traj.all_cons_satisfied))

Constraint satisfied : [ True  True]


the .all_cons_satisfied attribute **returns one boolean per trajectory stored!** You can also assess each constraint separately by accessing the attributes :
- .q_cons_satisfied
- .qdot_cons_satisfied
- .qddot_cons_satisfied
- .tau_cons_satisfied